# Setup

Este notebook nos va a servir como baseline para el resto de los notebooks sobre los que trabajaremos durante el módulo.

Está pensado como un punto de partida para poder instalar las librerías necesarias para poder trabajar con los notebooks que se irán desarrollando a lo largo del módulo.

In [2]:
import sys

print(sys.executable)

/opt/conda/bin/python


In [3]:
!python -m pip install --disable-pip-version-check -q pip --upgrade > /dev/null
!python -m pip install --disable-pip-version-check -q wrapt --upgrade > /dev/null

Keyring is skipped due to an exception: 'keyring.backends'
Keyring is skipped due to an exception: 'keyring.backends'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.4.0 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.29.8 which is incompatible.


In [4]:
!python -m pip install \
    --disable-pip-version-check \
    --quiet \
    --extra-index-url https://download.pytorch.org/whl/cpu \
    awscli \
    awswrangler \
    boto3 \
    botocore \
    tabulate \
    matplotlib==3.5.3 \
    psycopg2-binary==2.9.5 \
    PyAthena==2.14.0 \
    sagemaker-experiments==0.1.39 \
    sagemaker==2.116.0 \
    seaborn==0.12.1 \
    scikit-learn==1.0.2 \
    smdebug==1.0.12 \
    SQLAlchemy==1.4.43 \
    stepfunctions==2.3.0 \
    tensorflow==2.9.2 \
    torch==1.13.0 \
    torch-model-archiver==0.6.0 \
    torchaudio==0.13.0 \
    torchserve==0.6.0 \
    torchvision==0.14.0 \
    transformers==4.24.0

Keyring is skipped due to an exception: 'keyring.backends'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
aiobotocore 2.4.0 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.29.8 which is incompatible.


In [5]:
!conda install --quiet --yes zip

Solving environment: ...working... The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.
done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - zip


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.10.11 |       h06a4308_0         124 KB
    certifi-2022.9.24          |   py37h06a4308_0         154 KB
    conda-22.9.0               |   py37h06a4308_0         878 KB
    zip-3.0                    |       h7f8727e_1         111 KB
    ------------------------------------------------------------
                                           Total:         1.2 MB

The following NEW packages will be INSTALLED:

  zip                pkgs/main/linux-64::zip-3.0-h7f8727e_1 None

The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-cert

In [16]:
setup_dependencies_passed = True

### Estableciendo permisos

In [6]:
import boto3
import botocore.exceptions
import sagemaker
from botocore.config import Config

config = Config(retries={"max_attempts": 10, "mode": "adaptive"})
iam = boto3.client("iam", config=config)

admin = False

role_arn = sagemaker.get_execution_role()
print(f"Role ARN: {role_arn}")

role_name = role_arn.split("/")[-1]
print(f"Role name: {role_name}")

Role ARN: arn:aws:iam::688013747199:role/service-role/AmazonSageMaker-ExecutionRole-20221124T192018
Role name: AmazonSageMaker-ExecutionRole-20221124T192018


In [7]:
required_policies = [
    "AdministratorAccess",
    "IAMFullAccess",
#     "SecretsManagerReadWrite",
#     "AmazonS3FullAccess",
#     "AmazonAthenaFullAccess",
#     "ComprehendFullAccess",
#     "AmazonEC2ContainerRegistryFullAccess",
#     "AmazonRedshiftFullAccess",
#     "AWSStepFunctionsFullAccess",
#     "AmazonSageMakerFullAccess",
#     "AmazonKinesisFullAccess",
#     "AmazonKinesisFirehoseFullAccess",
#     "AmazonKinesisAnalyticsFullAccess",
]

Vamos a obtener la diferencia entre los dos set de Python, de tal forma que tengamos una lista de aquellas policies requeridas que no estén actualmente
asociadas a nuestro rol de SageMaker. Para ver este fragmento de código, podemos descomentar cualquiera de las policies de arriba y ejecutar los siguiente

In [8]:
current_policies = ["AdministratorAccess"]
print(list(set(required_policies).difference(current_policies)))

['IAMFullAccess']


Es importante tener en cuenta que para poder trabajar con IAM, nuestro rol tiene que tener asociada previamente la policy "IAMFullAccess". Así mismo,
hay que ser conscientes de que esto es un curso, y que en un entorno productivo tenemos que seguir el principio del mínimo privilegio.

In [27]:
try:
    current_policies = [policy['PolicyName'] for policy in iam.list_attached_role_policies(RoleName=role_name)["AttachedPolicies"]]

    for policy in list(set(required_policies).difference(current_policies)):
        response = iam.attach_role_policy(
            PolicyArn=f"arn:aws:iam::aws:policy/{policy}",RoleName=role_name
        )
        print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
    setup_iam_roles_passed = True

except botocore.exceptions.ClientError as error:
    print(f"""
        **[ERROR]
        SageMakerExecutionRole needs the AdministratorAccess or IAMFullAccess policy attached.
        **"
    """)
    raise error

except Exception as error:
    raise error

Policy AdministratorAccess has been succesfully attached to role: sagemaker-vanilla-SageMakerExecutionRole-A9Y6X9M9UTWM


## Guardando variables en el store de Jupyter

Con este paso nos aseguramos de que si alguien ejecuta nuestro notebook, tenga un entorno seguro

[Documentación sobre storemagic](https://ipython.readthedocs.io/en/stable/config/extensions/storemagic.html)

In [28]:
%store setup_dependencies_passed
%store setup_iam_roles_passed
%store role_arn
%store role_name

Stored 'setup_dependencies_passed' (bool)
Stored 'setup_iam_roles_passed' (bool)
Stored 'role_arn' (str)
Stored 'role_name' (str)


In [29]:
%store

Stored variables and their in-db values:
role_arn                              -> 'arn:aws:iam::688013747199:role/sagemaker-vanilla-
role_name                             -> 'sagemaker-vanilla-SageMakerExecutionRole-A9Y6X9M9
setup_dependencies_passed             -> True
setup_iam_roles_passed                -> True


## Liberando recursos AWS

In [30]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>

In [31]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.b>p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernelbutton>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
script>